In [1]:
from resolve import resolve 
resolve()
from widgets import dashboard

username = input("Please enter your username: ")
dashboard.run(username)

Please enter your username:  test-user
Enter your API key:  ········


User: test-user has been successfully authenticated!


Button(description='Run', style=ButtonStyle())

Button(description='Cancel', disabled=True, style=ButtonStyle())

Output()

Output()